In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)
            
            if train:
                Y_f = np.array(list(map(lambda x:np.sum(np.sum(x, axis=0), axis=0), Y_batch)))
                Y_f[Y_f < 25] = 0
                Y_f[Y_f >= 25] = 1
                
            
                yield (X_batch, Y_f)
            else:
                yield X_batch

structure_ids = []   
for line in open('./structures lists/stucture ids homo sapiens.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)
for line in open('./structures lists/stucture ids synthetic construct.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)
for line in open('./structures lists/stucture ids virus.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)

structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

num_test = int(len(structure_ids)*0.3)
pp_train = 0
pp_test = 0
pos = 0
neg = 0
for i, structure_id in enumerate(structure_ids):
    rna = np.load('../data/voxelized data 14x14x17 2/' + structure_id + '_rna_3D.npy', mmap_mode='r')

    pp = 0
    p = 0
    n = 0
    for bb in rna:
        n_nucleotides = np.sum(bb)
        if n_nucleotides >= 100:
            pp +=1
        if 100 > n_nucleotides > 0:
            p +=1
        if n_nucleotides == 0:
            n +=1
    pos +=p
    neg +=n
    if i <= num_test:
        pp_test +=pp
    else:
        pp_train +=pp

proc_test = pp_test/(pp_test+pp_train)
pos_test = int(pos*proc_test)
neg_test = int(neg*proc_test)

print(pp_train, pp_test, (pp_train + pp_test), pos, neg)
print(proc_test, pos_test, neg_test)

X_train = []
X_test = []
Y_train = []
Y_test = []
num_aa_train = 0
num_aa_test = 0
num_pp = 0
num_p = 0
num_n = 0
for j, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 14x14x17 2/' + structure_id + '_protein.npy', mmap_mode='r')
    rna = np.load('../data/voxelized data 14x14x17 2/' + structure_id + '_rna_3D.npy', mmap_mode='r')

    k = 0
    pp_ = 0
    while (np.sum(rna[k]) >= 100) and (k < len(rna)-1):
        pp_ +=1
        k +=1

    if j <= num_test:
        X_test.extend(protein[:pp_, :, :, :, :3])
        Y_test.extend(rna[:pp_])
        num_aa_test +=pp_
        num_pp +=pp_

    else:
        X_train.extend(protein[:pp_, :, :, :, :3])
        Y_train.extend(rna[:pp_])
        num_aa_train +=pp_

    p_= 0 
    while (100 > np.sum(rna[k]) > 0)  and (k < len(rna)-1):
        if (pos_test > 0):
            p_ +=1
            pos_test -=1
        k +=1


    X_test.extend(protein[pp_:(pp_+p_), :, :, :, :3])
    Y_test.extend(rna[pp_:(pp_+p_)])
    num_aa_test +=p_
    num_p +=p_

    n_= 0
    while (np.sum(rna[k]) == 0) and (neg_test > 0) and (k < len(rna)-1):
        n_ +=1
        neg_test -=1
        k +=1

    X_test.extend(protein[(pp_+p_):(pp_+p_+n_), :, :, :, :3])
    Y_test.extend(rna[(pp_+p_):(pp_+p_+n_)])
    num_aa_test +=n_
    num_n +=n_


Y_test_ = np.array(list(map(lambda x:np.sum(np.sum(x, axis=0), axis=0), Y_test)))
Y_test_[Y_test_ < 25] = 0
Y_test_[Y_test_ >= 25] = 1


print(Y_test_.shape)

n_steps_train = int(num_aa_train/1024) 
n_steps_test = int(num_aa_test/1024)

print(num_aa_train, num_aa_test)

print(pp_train, pp_test, pos_test, neg_test)
print(num_pp, num_p, num_n)

generator_train = generator(X_train, Y_train, 1024, True)
generator_validation = generator(X_test, Y_test, 1024, True)
generator_test = generator(X_test, Y_test, 1024, False)

ins = tf.keras.layers.Input((14, 14, 17, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(maxp3)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=17, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy', 'mse'])

model.summary()

# checkpoint
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, mode='min', min_delta=0.0001)
# mc = tf.keras.callbacks.ModelCheckpoint("weights_best.hdf5", monitor='val_loss', verbose=0, save_best_only=True, mode='min')

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
history = model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, 
                    validation_data = generator_validation, validation_steps=n_steps_test, callbacks=[es], verbose=1)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss')


# model_best = model
# model_best.load_weights("weights_best.hdf5")
# print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
# model_best.save('model_cnn_15_2.h5')
# Y_pred = model_best.predict(X_test, batch_size=200)
Y_pred_ = model.predict_generator(generator_test, steps=n_steps_test)
# print(Y_pred.shape)
# Y_pred_ = np.array([Y_pred[i].reshape((2,2,17)) for i in range(Y_pred.shape[0])])

#CNN
Y_pred_prob = Y_pred_
Y_pred_[Y_pred_ >= 0.5] = 1
Y_pred_[Y_pred_ < 0.5] = 0

print(Y_pred_.shape)
print(Y_test_.shape)
Y_test_ = Y_test_[:Y_pred_.shape[0]]


print('CNN: \n')

confusion_matrix = [sklearn.metrics.confusion_matrix(Y_test_[:,i], Y_pred_[:, i]) for i in range(17)]
accuracy = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix]
auc = [sklearn.metrics.roc_auc_score(Y_test_[:,i], Y_pred_prob[:,i]) for i in range(17)]

for q in range(17):
    print(f'level {q+1}')
    print(confusion_matrix[q], np.round(accuracy[q], 2), np.round(auc[q], 2))

# baseline model
# predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
Y_pred_base = np.zeros(Y_test_.shape)

# po = np.sum(Y_train, axis=0)/Y_train.shape[0]
# po[po >= 0.5] = 1
# po[po < 0.5] = 0
# Y_pred_base = np.tile(po, (Y_test.shape[0],1))

print(Y_pred_base.shape)
print(f'\n BASELINE MODEL: \n')

confusion_matrix_base = [sklearn.metrics.confusion_matrix(Y_test_[:,i], Y_pred_base[:,i]) for i in range(17)]
accuracy_base = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_base]
auc_base = [sklearn.metrics.roc_auc_score(Y_test_[:, i], Y_pred_base[:,i]) for i in range(17)]

for q in range(17):
    print(f'level {q+1}')
    print(confusion_matrix_base[q], np.round(accuracy_base[q], 2), np.round(auc_base[q], 2))

#random model
Y_pred_random = np.random.random(Y_test_.shape)
Y_pred_random[Y_pred_random >= 0.5] = 1
Y_pred_random[Y_pred_random < 0.5] = 0

print(f'\n RANDOM MODEL: \n')

confusion_matrix_random = [sklearn.metrics.confusion_matrix(Y_test_[:,i], Y_pred_random[:,i]) for i in range(17)]
accuracy_random = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_random]
auc_random = [sklearn.metrics.roc_auc_score(Y_test_[:,i], Y_pred_random[:,i]) for i in range(17)]

for q in range(17):
    print(f'level {q+1}')
    print(confusion_matrix_random[q], np.round(accuracy_random[q], 2), np.round(auc_random[q], 2))

549
24676 13143 37819 94223 334320
0.3475237314577329 32744 116184
(162070, 17)
24673 162070
24676 13143 0 0
13142 32744 116184
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14, 14, 17, 3)     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 14, 14, 17, 64)    5248      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 14, 14, 17, 32)    55328     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 14, 14, 17, 32)    27680     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 7, 8, 32)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 7, 8, 32)       27680     
______________

Epoch 29/500
24/24 [==============================] - 47s 2s/step - loss: 0.4159 - acc: 0.8070 - mean_squared_error: 0.1344 - val_loss: 0.3726 - val_acc: 0.8944 - val_mean_squared_error: 0.1044
Epoch 30/500
24/24 [==============================] - 48s 2s/step - loss: 0.4117 - acc: 0.8102 - mean_squared_error: 0.1323 - val_loss: 0.3200 - val_acc: 0.9322 - val_mean_squared_error: 0.0828
Epoch 31/500
24/24 [==============================] - 48s 2s/step - loss: 0.3964 - acc: 0.8184 - mean_squared_error: 0.1272 - val_loss: 0.4432 - val_acc: 0.8876 - val_mean_squared_error: 0.1252
Epoch 32/500
24/24 [==============================] - 47s 2s/step - loss: 0.4053 - acc: 0.8125 - mean_squared_error: 0.1305 - val_loss: 0.3765 - val_acc: 0.8951 - val_mean_squared_error: 0.1039
Epoch 33/500
24/24 [==============================] - 48s 2s/step - loss: 0.3890 - acc: 0.8223 - mean_squared_error: 0.1245 - val_loss: 0.3959 - val_acc: 0.8857 - val_mean_squared_error: 0.1123
Epoch 34/500
24/24 [==========

24/24 [==============================] - 47s 2s/step - loss: 0.2543 - acc: 0.8915 - mean_squared_error: 0.0781 - val_loss: 0.2584 - val_acc: 0.9225 - val_mean_squared_error: 0.0653
Epoch 72/500
24/24 [==============================] - 47s 2s/step - loss: 0.2456 - acc: 0.8961 - mean_squared_error: 0.0751 - val_loss: 0.3720 - val_acc: 0.8848 - val_mean_squared_error: 0.0960
Epoch 73/500
24/24 [==============================] - 47s 2s/step - loss: 0.2476 - acc: 0.8953 - mean_squared_error: 0.0758 - val_loss: 0.3718 - val_acc: 0.8877 - val_mean_squared_error: 0.0943
Epoch 74/500
24/24 [==============================] - 47s 2s/step - loss: 0.2538 - acc: 0.8922 - mean_squared_error: 0.0779 - val_loss: 0.3652 - val_acc: 0.8925 - val_mean_squared_error: 0.0909
Epoch 75/500
24/24 [==============================] - 48s 2s/step - loss: 0.2535 - acc: 0.8933 - mean_squared_error: 0.0775 - val_loss: 0.2746 - val_acc: 0.9212 - val_mean_squared_error: 0.0673
Epoch 76/500
24/24 [=======================

24/24 [==============================] - 48s 2s/step - loss: 0.2077 - acc: 0.9131 - mean_squared_error: 0.0631 - val_loss: 0.5320 - val_acc: 0.8046 - val_mean_squared_error: 0.1439
Epoch 114/500
24/24 [==============================] - 48s 2s/step - loss: 0.1958 - acc: 0.9191 - mean_squared_error: 0.0591 - val_loss: 0.3730 - val_acc: 0.8810 - val_mean_squared_error: 0.0938
Epoch 115/500
24/24 [==============================] - 48s 2s/step - loss: 0.1906 - acc: 0.9207 - mean_squared_error: 0.0577 - val_loss: 0.3804 - val_acc: 0.8690 - val_mean_squared_error: 0.1001
Epoch 116/500
24/24 [==============================] - 48s 2s/step - loss: 0.1850 - acc: 0.9237 - mean_squared_error: 0.0557 - val_loss: 0.3675 - val_acc: 0.8720 - val_mean_squared_error: 0.0982
Epoch 117/500
24/24 [==============================] - 48s 2s/step - loss: 0.1943 - acc: 0.9199 - mean_squared_error: 0.0586 - val_loss: 0.5428 - val_acc: 0.7542 - val_mean_squared_error: 0.1683
Epoch 118/500
24/24 [==================

24/24 [==============================] - 48s 2s/step - loss: 0.1626 - acc: 0.9326 - mean_squared_error: 0.0489 - val_loss: 0.3805 - val_acc: 0.8783 - val_mean_squared_error: 0.0947
Epoch 156/500
24/24 [==============================] - 48s 2s/step - loss: 0.1683 - acc: 0.9312 - mean_squared_error: 0.0506 - val_loss: 0.3498 - val_acc: 0.8917 - val_mean_squared_error: 0.0852
Epoch 157/500
24/24 [==============================] - 48s 2s/step - loss: 0.1643 - acc: 0.9321 - mean_squared_error: 0.0495 - val_loss: 0.3457 - val_acc: 0.8868 - val_mean_squared_error: 0.0875
Epoch 158/500
24/24 [==============================] - 48s 2s/step - loss: 0.1612 - acc: 0.9336 - mean_squared_error: 0.0485 - val_loss: 0.3163 - val_acc: 0.8970 - val_mean_squared_error: 0.0800
Epoch 159/500
24/24 [==============================] - 48s 2s/step - loss: 0.1696 - acc: 0.9299 - mean_squared_error: 0.0511 - val_loss: 0.4530 - val_acc: 0.8450 - val_mean_squared_error: 0.1173
Epoch 160/500
24/24 [==================

level 1
[[156304      0]
 [  5488      0]] 0.97 0.5
level 2
[[156791      0]
 [  5001      0]] 0.97 0.5
level 3
[[156905      0]
 [  4887      0]] 0.97 0.5
level 4
[[156845      0]
 [  4947      0]] 0.97 0.5
level 5
[[156614      0]
 [  5178      0]] 0.97 0.5
level 6
[[156135      0]
 [  5657      0]] 0.97 0.5
level 7
[[155486      0]
 [  6306      0]] 0.96 0.5
level 8
[[155093      0]
 [  6699      0]] 0.96 0.5
level 9
[[154637      0]
 [  7155      0]] 0.96 0.5
level 10
[[154517      0]
 [  7275      0]] 0.96 0.5
level 11
[[154585      0]
 [  7207      0]] 0.96 0.5
level 12
[[154734      0]
 [  7058      0]] 0.96 0.5
level 13
[[154966      0]
 [  6826      0]] 0.96 0.5
level 14
[[155267      0]
 [  6525      0]] 0.96 0.5
level 15
[[155543      0]
 [  6249      0]] 0.96 0.5
level 16
[[155892      0]
 [  5900      0]] 0.96 0.5
level 17
[[156278      0]
 [  5514      0]] 0.97 0.5

 RANDOM MODEL: 

level 1
[[78204 78100]
 [ 2773  2715]] 0.5 0.5
level 2
[[78562 78229]
 [ 2472  2529]] 0.5 